In [3]:
conda install pandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/bcchochoao/opt/anaconda2/envs/Notebook

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    mkl-service-2.3.0          |   py38hfbe908c_0          42 KB
    mkl_fft-1.0.15             |   py38h5e564d8_0         138 KB
    mkl_random-1.1.0           |   py38h6440ff4_0           5 KB
    numpy-1.18.1               |   py38h7241aed_0           5 KB
    numpy-base-1.18.1          |   py38h6575580_1         3.9 MB
    pandas-1.0.3               |   py38h6c726b0_0         7.9 MB
    ------------------------------------------------------------
                                           Total:        12.0

In [6]:
# Downgraded to the previous version due to the critical bug in 0.16
!pip install catboost==0.15

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [18]:
import pandas as pd
import numpy as np

import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve
from catboost import Pool, CatBoostClassifier

from scipy.stats import pearsonr, chi2_contingency
from itertools import combinations
from statsmodels.stats.proportion import proportion_confint

ModuleNotFoundError: No module named 'sklearn'

In [17]:
!pip install sklearn

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=71f9c1d28b41b5198d6aeabf74b84606511c6e5577dd10e98d666ad18f77a7b0
  Stored in directory: /Users/bcchochoao/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [10]:
conda install seaborn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/bcchochoao/opt/anaconda2/envs/Notebook

  added / updated specs:
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scipy-1.4.1                |   py38h44e99c9_0        13.0 MB
    seaborn-0.10.0             |             py_0         163 KB
    ------------------------------------------------------------
                                           Total:        13.1 MB

The following NEW packages will be INSTALLED:

  scipy              pkgs/main/osx-64::scipy-1.4.1-py38h44e99c9_0
  seaborn            pkgs/main/noarch::seaborn-0.10.0-py_0



scipy-1.4.1          | 13.0 MB   | ##################################### | 100%

In [8]:
conda install matplotlib

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/bcchochoao/opt/anaconda2/envs/Notebook

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cycler-0.10.0              |           py38_0          14 KB
    kiwisolver-1.0.1           |   py38h0a44026_0          58 KB
    matplotlib-3.1.3           |           py38_0          21 KB
    matplotlib-base-3.1.3      |   py38h9aa3819_0         4.9 MB
    pyparsing-2.4.7            |             py_0          65 KB
    ------------------------------------------------------------
                                           Total:         5.1 MB

The following NEW packages will be INSTALLED:

  cycler 